**Assignment:2**

# **Work by:** Saurabh Annadate.


In [78]:
import numpy as np
import random
import pandas as pd

1) Loading data.

2) Making a function which will return the pixel values and corresponding digit in X,Y format

In [79]:
def load_data(path='https://raw.githubusercontent.com/seaborncat/ML-Project/master/mnist_train.csv'): #github link to the training data provided
    f = pd.read_csv(path)
    tr_x=np.array(f.iloc[:,:-1])
    tr_y=np.array(f.iloc[:,-1])
    X = [np.reshape(x, (784, 1))/255 for x in tr_x]      # X matrix of training dataset
    Y = [vectorized_result(y) for y in tr_y]            # Y matrix of training dataset (hot-encoded instead of real numbers)
    return (X,Y)                                        #it will be easier to handle hot-encoded,we can output real numbers also if we want
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [80]:
X,Y=load_data()

**Building the neural network of zero hidden layers...**

784>10 

1)Minimization function used here is cross-entropy.

2)Output neurons are based on softmax function.


In [81]:
def update_weights_perceptrons( X,Y,weights,biases, eta): 
        mini_batch_size=10000 #mini_batch_size for stochastic GD (here taken complete data for this example in a single step)
        training_data = list(zip(X,Y))
        n = len(training_data)
        random.shuffle(training_data)
        mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
        for mini_batch in mini_batches:           #updates weights and biases here..
            grad_b = [np.zeros(b.shape) for b in biases]
            grad_w = [np.zeros(w.shape) for w in weights]
            for x, y in mini_batch:
                delta_grad_b, delta_grad_w = backprop(x, y,weights,biases)
                grad_b = [nb+dnb for nb, dnb in zip(grad_b, delta_grad_b)]
                grad_w = [nw+dnw for nw, dnw in zip(grad_w, delta_grad_w)]
            weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(weights, grad_w)]
            biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(biases, grad_b)]
            
        return weights,biases

def backprop( x, y,weights,biases):
    grad_b = [np.zeros(b.shape) for b in biases]  #grad_b and grad_w are small steps towards minima of b and w
    grad_w = [np.zeros(w.shape) for w in weights]
    # feedforward
    activation = x
    activations = [x]            # list to store all the activations, layer by layer
    zs = []                      # list to store all the z vectors, layer by layer
    for b, w in zip(biases, weights):
        z = np.dot(w, activation)+b
        zs.append(z)
        activation = softmax(z)
        activations.append(activation)
        #backprop starts 
        delta = delta1(zs[-1], activations[-1], y)  #first argument tells the layer number(last layer in this case)
        grad_b[-1] = delta
        grad_w[-1] = np.dot(delta, activations[-2].transpose())
        
    return (grad_b, grad_w)

    

#extra functions used
def softmax(X):
    exps = np.exp(X)
    return exps / np.sum(exps)
def fn(a, y):
    return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a))) #crossentropy 
def delta1(z, a, y): #this is an error between actual value and last activation
    return (a-y)


In [82]:
sizes=np.array([784,10])   #initializing weights and biases for given layers
biases = [np.random.randn(y, 1) for y in sizes[1:]]
weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]


In [83]:
w,b=update_weights_perceptrons(X,Y,weights,biases,1)

In [84]:
print(w)
print(b)

[array([[ 1.76854653,  1.53138716, -0.38072967, ...,  0.3148166 ,
         0.45139251,  0.1935604 ],
       [-0.65682131,  0.51557157,  1.50927265, ..., -2.01046924,
        -0.44023945,  0.04889727],
       [-0.41455056, -0.91405244,  1.40586892, ...,  0.33076941,
        -0.08462456,  1.58111736],
       ...,
       [-0.78385053,  0.47677477,  1.1413737 , ...,  0.07966108,
        -0.84374008,  0.53009183],
       [ 0.31945697,  0.93940009, -1.87631725, ..., -2.39040426,
        -0.59044523, -1.61732873],
       [-0.89357978, -0.18336752, -0.35836683, ..., -0.39673404,
         0.27951699, -0.67444063]])]
[array([[-0.83758476],
       [ 0.39065983],
       [-0.52946842],
       [-0.87705436],
       [-0.3645372 ],
       [-0.33686772],
       [ 1.75075929],
       [ 1.25311792],
       [ 0.82435307],
       [-0.14928285]])]


In [85]:
#let's define all the functions we'll use in future here..
def softmax(X):
    exps = np.exp(X)
    return exps / np.sum(exps)
def fn(a, y):
    return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))
def delta1(z, a, y):
    return (a-y)
def softmax_grad(s): 
    jacobian_m = np.diag(s)   
    for i in range(len(jacobian_m)):
        for j in range(len(jacobian_m)):
            if i == j:
                jacobian_m[i][j] = s[i] * (1-s[i])
            else: 
                jacobian_m[i][j] = -s[i]*s[j]
    return jacobian_m
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))
def tanh(z):
    return np.tanh(z) 
def tanh_grad(z):
    return 1-np.square(np.tanh(z))
def relu(z, deriv = False):   #fixed relu (will not blow ar z=0/1 because changed to z=0.0001/0.999)
    if not deriv:
        relud = z/255
        relud[relud < 0] = 0
        return relud
    deriv = z
    deriv[deriv <= 0] = 0
    deriv[deriv > 0] = 1
    return deriv


Neural Network with 1 hidden layer...

**784>10>10**

hidden layer uses sigmoid as activation function

In [86]:
def update_weights_single_layer( X,Y,weights,biases, eta): 
        ''' Using mini batches in gradient descent from now
        because its computationally efficient and takes lesser time'''
        mini_batch_size=10 
        training_data = list(zip(X,Y))
        n = len(training_data)
        random.shuffle(training_data)
        mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
        for mini_batch in mini_batches:
            grad_b = [np.zeros(b.shape) for b in biases]
            grad_w = [np.zeros(w.shape) for w in weights]
            for x, y in mini_batch:
                delta_grad_b, delta_grad_w = backprop(x, y,weights,biases)
                grad_b = [nb+dnb for nb, dnb in zip(grad_b, delta_grad_b)]
                grad_w = [nw+dnw for nw, dnw in zip(grad_w, delta_grad_w)]
            weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(weights, grad_w)]
            biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(biases, grad_b)]
            
        return weights,biases

def backprop( x, y,weights,biases):
    grad_b = [np.zeros(b.shape) for b in biases]  #grad_b and grad_w are small steps towards minima of b and w
    grad_w = [np.zeros(w.shape) for w in weights]
    # feedforward
    activation = x
    activations = [x] # list to store all the activations, layer by layer
    zs = [] # list to store all the z vectors, layer by layer
    
    for b, w in zip(biases[:-1], weights[:-1]):  #activations upto second last layer using sigmoid
        z = np.dot(w, activation)+b
        zs.append(z)
        activation = sigmoid(z)
        activations.append(activation)     
    for b, w in zip(biases[-1:], weights[-1:]): #activation of the last layer using softmax  
        z = np.dot(w, activation)+b
        zs.append(z)
        activation = softmax(z)
        activations.append(activation)
    delta = delta1(zs[-1], activations[-1], y)     #backprop from softmax layer
    grad_b[-1] = delta
    grad_w[-1] = np.dot(delta, activations[-2].transpose())
    for l in range(2, len(biases)+1):              #backprop from  rest of the layer
        z = zs[-l]
        sp = sigmoid_prime(z)
        delta = np.dot(weights[-l+1].transpose(), delta) *sp  
        grad_b[-l] = delta
        grad_w[-l] = np.dot(delta, activations[-l-1].transpose())
    return (grad_b, grad_w)



In [87]:
sizes1=np.array([784,10,10])
biases1 = [np.random.randn(y, 1) for y in sizes1[1:]]
weights1 = [np.random.randn(y, x) for x, y in zip(sizes1[:-1], sizes1[1:])]



In [88]:
w1,b1=update_weights_single_layer(X,Y,weights1,biases1,1)
print(w1)
print(b1)

[array([[-0.19558433,  1.5947903 , -0.69658389, ..., -2.43648752,
         1.07612152, -1.48686689],
       [-0.54567876,  0.67134208,  0.29820422, ...,  0.86704487,
        -1.60651281, -0.70651083],
       [-1.5870352 , -0.68850767, -0.08476187, ..., -1.43377292,
         0.58210878, -1.41604227],
       ...,
       [ 0.86605891,  0.81242171,  1.8300582 , ...,  0.44594629,
        -0.81481453, -1.7043912 ],
       [-0.15025627,  1.54554692,  0.08987386, ..., -0.61524367,
         0.00887965, -0.96974795],
       [ 0.09670126, -0.81654878, -0.23200942, ..., -1.0191568 ,
        -0.68041648, -0.37926725]]), array([[ 0.52807677, -2.93518663, -1.97061199,  1.758346  , -1.36450569,
        -1.81060278,  4.08901211, -1.04289101,  2.23053977, -1.6761527 ],
       [-1.38379237,  1.62691553, -1.74055665,  2.21597915,  1.4542596 ,
         2.19449691, -1.11558785, -2.57081136, -2.36026424,  2.32183524],
       [-0.9307489 , -1.4468928 ,  0.02871845, -0.90373804,  0.93616759,
        -0.6314634

starting Neural Network with 2 hidden layers...

**784>10>10>10**

In [89]:
def update_weights_double_layer( X,Y,weights,biases, eta): 
        mini_batch_size=10 #mini_batch_size for stochastic GD
        training_data = list(zip(X,Y))
        n = len(training_data)
        random.shuffle(training_data)
        mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
        for mini_batch in mini_batches:
            grad_b = [np.zeros(b.shape) for b in biases]
            grad_w = [np.zeros(w.shape) for w in weights]
            for x, y in mini_batch:
                delta_grad_b, delta_grad_w = backprop(x, y,weights,biases)
                grad_b = [nb+dnb for nb, dnb in zip(grad_b, delta_grad_b)]
                grad_w = [nw+dnw for nw, dnw in zip(grad_w, delta_grad_w)]
            weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(weights, grad_w)]
            biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(biases, grad_b)]
            
        return weights,biases

def backprop( x, y,weights,biases):
    grad_b = [np.zeros(b.shape) for b in biases]  
    grad_w = [np.zeros(w.shape) for w in weights]
    # feedforward
    activation = x
    activations = [x] # list to store all the activations, layer by layer
    zs = [] # list to store all the z vectors, layer by layer
    
    for b, w in zip(biases[:-1], weights[:-1]):
        z = np.dot(w, activation)+b
        zs.append(z)
        activation = sigmoid(z)
        activations.append(activation)
    for b, w in zip(biases[-1:], weights[-1:]):
        z = np.dot(w, activation)+b
        zs.append(z)
        activation = softmax(z)
        activations.append(activation)
    delta = delta1(zs[-1], activations[-1], y)
    grad_b[-1] = delta
    grad_w[-1] = np.dot(delta, activations[-2].transpose())
    for l in range(2, len(biases)+1):
        z = zs[-l]
        sp = sigmoid_prime(z)
        delta = np.dot(weights[-l+1].transpose(), delta) *sp
        grad_b[-l] = delta
        grad_w[-l] = np.dot(delta, activations[-l-1].transpose())
    return (grad_b, grad_w)


In [90]:
sizes2=np.array([784,10,10,10])
biases2 = [np.random.randn(y, 1) for y in sizes2[1:]]
weights2 = [np.random.randn(y, x) for x, y in zip(sizes2[:-1], sizes2[1:])]

In [91]:
w2,b2=update_weights_double_layer(X,Y,weights2,biases2,eta= 1)
print(w2)
print(b2)

[array([[-1.01925631,  0.16662307, -1.25545279, ...,  0.06015003,
         1.14296297,  0.63086857],
       [-1.11716206,  0.47671476, -0.53007367, ..., -1.04651319,
         0.60643914,  1.63261621],
       [-1.72321606, -0.14129194,  0.56378852, ...,  1.19883135,
         0.61797649,  0.20180285],
       ...,
       [-1.09939491,  0.6834544 ,  0.14031456, ...,  0.59316229,
        -1.08269484, -0.7953746 ],
       [-0.41263092, -1.63612621,  0.88505431, ...,  0.60924994,
        -0.38211105,  1.08987879],
       [ 0.43019252, -0.48654202, -0.07345832, ...,  0.19701464,
        -0.04862068, -0.14561675]]), array([[ 0.03868441,  1.33759203, -1.5504352 , -0.90080828, -1.33621513,
        -1.03490559, -0.05045638,  0.0552776 , -2.36018259, -1.40645622],
       [ 2.11455707,  3.20961277,  0.27198415, -1.42177998,  0.37740354,
         0.70193021, -1.49024721,  1.75290728, -0.55593166, -4.37744969],
       [-0.96739501, -1.20845311,  1.34754281, -2.45032335,  0.37433583,
        -2.9641175

Neural Network with 2 hidden layers with choices for choosing **activation function...**

784>10>10>10

Activation functions introduced:

1) tanh

2) relu

3) sigmoid

In [92]:
def update_weights_double_layer_act( X,Y,weights,biases, eta,activation): 
    mini_batch_size=10
    training_data = list(zip(X,Y))
    n = len(training_data)
    random.shuffle(training_data)
    mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
    for mini_batch in mini_batches:
        grad_b = [np.zeros(b.shape) for b in biases]
        grad_w = [np.zeros(w.shape) for w in weights]
        for x, y in mini_batch:
            delta_grad_b, delta_grad_w = backprop(x, y,weights,biases,activation)
            grad_b = [nb+dnb for nb, dnb in zip(grad_b, delta_grad_b)]
            grad_w = [nw+dnw for nw, dnw in zip(grad_w, delta_grad_w)]
        weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(weights, grad_w)]
        biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(biases, grad_b)]

    return weights,biases

def backprop( x, y,weights,biases,act):

    grad_b = [np.zeros(b.shape) for b in biases]
    grad_w = [np.zeros(w.shape) for w in weights]
    # feedforward
    activation = x
    activations = [x] # list to store all the activations, layer by layer
    zs = [] # list to store all the z vectors, layer by layer
    if act=='tanh':
        for b, w in zip(biases[:-1], weights[:-1]):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = tanh(z)
            activations.append(activation)
        for b, w in zip(biases[-1:], weights[-1:]):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = softmax(z)
            activations.append(activation)
        # backward pass
        delta = delta1(zs[-1], activations[-1], y)
        grad_b[-1] = delta
        grad_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, len(biases)+1):
            z = zs[-l]
            tp = tanh_grad(z)
            delta = np.dot(weights[-l+1].transpose(), delta)*tp 
            grad_b[-l] = delta
            grad_w[-l] = np.dot(delta, activations[-l-1].transpose())
    if act=='relu':
        for b, w in zip(biases[:-1], weights[:-1]):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = relu(z)
            activations.append(activation)
        for b, w in zip(biases[-1:], weights[-1:]):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = softmax(z)
            activations.append(activation)
        # backward pass
        delta = delta1(zs[-1], activations[-1], y)
        grad_b[-1] = delta
        grad_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, len(biases)+1):
            z = zs[-l]
            rl = relu(z,deriv=True)
            delta = np.dot(weights[-l+1].transpose(), delta)*rl
            grad_b[-l] = delta
            grad_w[-l] = np.dot(delta, activations[-l-1].transpose())
    if act=='sigmoid':
        
    
        for b, w in zip(biases[:-1], weights[:-1]):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        for b, w in zip(biases[-1:], weights[-1:]):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = softmax(z)
            activations.append(activation)
        delta = delta1(zs[-1], activations[-1], y)
        grad_b[-1] = delta
        grad_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, len(biases)+1):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(weights[-l+1].transpose(), delta) *sp
            grad_b[-l] = delta
            grad_w[-l] = np.dot(delta, activations[-l-1].transpose())
        
    return (grad_b, grad_w)



In [93]:
sizes2=np.array([784,10,10,10])
biases2 = [np.random.randn(y, 1) for y in sizes2[1:]]
weights2 = [np.random.randn(y, x) for x, y in zip(sizes2[:-1], sizes2[1:])]

In [100]:
update_weights_double_layer_act(X,Y,weights2,biases2, 1,activation='relu') #(activation={'tanh','relu','sigmoid'})  

([array([[-0.24174226,  0.0165086 ,  0.63058535, ...,  0.94569143,
           1.22747527, -0.18461562],
         [-0.16716123, -1.57744561, -0.62000267, ...,  0.08415125,
           0.8489621 , -1.72600231],
         [ 0.22788913,  0.91846034, -0.39433575, ...,  1.34170934,
          -1.19661036,  0.39180757],
         ...,
         [ 0.84684417,  0.49222717, -0.88528257, ...,  1.14381777,
          -0.80802832, -0.67286737],
         [-0.60176144, -0.40534512, -0.07173232, ...,  2.57037263,
           0.37595739,  2.13928011],
         [-0.22286425,  1.24262112,  1.05213377, ..., -0.26653912,
          -1.49207114, -0.76708851]]),
  array([[ 36.99995584, -18.63396918,   0.09496369,  19.45472073,
          -15.57458329,   4.90061867, -18.17336142, -17.21602552,
            6.75949479,   2.28743959],
         [ -0.59432519,  17.41256297,  -0.45232943, -16.7699783 ,
            6.33573495, -22.53907753,   3.76105047,  14.38008856,
          -21.27112956,   0.96812334],
         [ 14.2345

Implementing **momentum** with gradient descent and adding **epochs** for multiple iterations... 

In [95]:
def update_weights_double_layer_mom(X,Y,weights,biases, eta,momentum,epochs,activation): #mini-batches for stochastic GD
    mini_batch_size=10
    beta=momentum
    training_data = list(zip(X,Y))
    n = len(training_data)
    for j in range (epochs):
        random.shuffle(training_data)
        mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
        for mini_batch in mini_batches:
            grad_b = [np.zeros(b.shape) for b in biases]
            grad_w = [np.zeros(w.shape) for w in weights]
            for x, y in mini_batch:
                delta_grad_b, delta_grad_w = backprop(x, y,weights,biases,activation)
                grad_b = [nb+dnb for nb, dnb in zip(grad_b, delta_grad_b)]
                grad_w = [nw+dnw for nw, dnw in zip(grad_w, delta_grad_w)]
            #momentum implementation
            #VdW = β x VdW + (1 – β) x dW
            vdw=0
            vdw = [vdw*beta+(1-beta)*nab_w for nab_w in grad_w]
            #Vdb = β x Vdb + (1 – β) x db
            vdb=0
            vdb = [vdb*beta+(1-beta)*nab_b for nab_b in grad_b]
            weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(weights, vdw)]
            biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(biases, vdb)]
        print("epoch:",j+1,"done")
    return weights,biases


def backprop( x, y,weights,biases,act):

    grad_b = [np.zeros(b.shape) for b in biases]
    grad_w = [np.zeros(w.shape) for w in weights]
    # feedforward
    activation = x
    activations = [x] # list to store all the activations, layer by layer
    zs = [] # list to store all the z vectors, layer by layer
    if act=='tanh':
        for b, w in zip(biases[:-1], weights[:-1]):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = tanh(z)
            activations.append(activation)
        for b, w in zip(biases[-1:], weights[-1:]):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = softmax(z)
            activations.append(activation)
        # backward pass
        delta = delta1(zs[-1], activations[-1], y)
        grad_b[-1] = delta
        grad_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, len(biases)+1):
            z = zs[-l]
            tp = tanh_grad(z)
            delta = np.dot(weights[-l+1].transpose(), delta)*tp 
            grad_b[-l] = delta
            grad_w[-l] = np.dot(delta, activations[-l-1].transpose())
    if act=='relu':
        for b, w in zip(biases[:-1], weights[:-1]):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = relu(z)
            activations.append(activation)
        for b, w in zip(biases[-1:], weights[-1:]):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = softmax(z)
            activations.append(activation)
        # backward pass
        delta = delta1(zs[-1], activations[-1], y)
        grad_b[-1] = delta
        grad_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, len(biases)+1):
            z = zs[-l]
            rl = relu(z,deriv=True)
            delta = np.dot(weights[-l+1].transpose(), delta)*rl
            grad_b[-l] = delta
            grad_w[-l] = np.dot(delta, activations[-l-1].transpose())
    if act=='sigmoid':
        
    
        for b, w in zip(biases[:-1], weights[:-1]):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        for b, w in zip(biases[-1:], weights[-1:]):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = softmax(z)
            activations.append(activation)
        delta = delta1(zs[-1], activations[-1], y)
        grad_b[-1] = delta
        grad_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, len(biases)+1):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(weights[-l+1].transpose(), delta) *sp
            grad_b[-l] = delta
            grad_w[-l] = np.dot(delta, activations[-l-1].transpose())
        
    return (grad_b, grad_w)


In [96]:
sizes2=np.array([784,10,10,10])
biases2 = [np.random.randn(y, 1) for y in sizes2[1:]]
weights2 = [np.random.randn(y, x) for x, y in zip(sizes2[:-1], sizes2[1:])]

In [101]:
update_weights_double_layer_mom(X,Y,weights2,biases2,eta=1,momentum=0.9,epochs=5,activation='relu')

epoch: 1 done
epoch: 2 done
epoch: 3 done
epoch: 4 done
epoch: 5 done


([array([[-0.24174226,  0.0165086 ,  0.63058535, ...,  0.94569143,
           1.22747527, -0.18461562],
         [-0.16716123, -1.57744561, -0.62000267, ...,  0.08415125,
           0.8489621 , -1.72600231],
         [ 0.22788913,  0.91846034, -0.39433575, ...,  1.34170934,
          -1.19661036,  0.39180757],
         ...,
         [ 0.84684417,  0.49222717, -0.88528257, ...,  1.14381777,
          -0.80802832, -0.67286737],
         [-0.60176144, -0.40534512, -0.07173232, ...,  2.57037263,
           0.37595739,  2.13928011],
         [-0.22286425,  1.24262112,  1.05213377, ..., -0.26653912,
          -1.49207114, -0.76708851]]),
  array([[ 1.09921856e+01, -8.09155609e+00, -2.19578888e+00,
           2.07486989e+01, -2.18110307e+01,  1.16706481e+00,
          -1.71062710e+00, -4.78602999e+00,  2.94889317e+01,
           1.94709364e+01],
         [ 5.79091719e+00,  8.15864520e+00,  8.30668292e+00,
           5.74722524e+00,  8.03600541e+00, -3.73458715e+00,
          -2.03795810e+00, 